In [44]:
import time
import cv2
import math

import numpy as np

from swarm import Swarm


In [45]:
NUM_DRONES = 3

AREA_SIZE_X = 50
AREA_SIZE_Y = 50
GRID_SIZE = 50           # Size of the Forest
grid_offset=GRID_SIZE/2

CAMERA_FOV_DEGREE = 50
camera_fov=np.deg2rad(CAMERA_FOV_DEGREE)
print(camera_fov)
IMAGE_SIZE = 512 #How many Scanning points each image has per Row (Images are 512x512)

MOVE_DISTANCE = 1       # How far a Drone can move each Timestep
DRONE_HEIGHT = 35

TIMESTEPS = 10
POPULATION_SIZE = 10
NUM_GENERATIONS = 10
MUTATION_RATE = 0.1


0.8726646259971648


In [46]:
camera_offset= 35 * np.tan(camera_fov/2)

In [50]:
#Create "Base Grid"
#by Calculating distance between each scanning point at DRONE_HEIGHT

def calculate_world_coordinates(drone_pos, image_radius, img_x, img_y):
    pos_x = drone_pos[0] + (img_x-256)/512 * image_radius * 2
    pos_y = drone_pos[1] + (img_y-256)/512 * image_radius * 2
    return (pos_x, pos_y)
print(calculate_world_coordinates(np.array([0,0]),16,512,512))
#def getSimplifiedConvexHull(waypoints):

sample_positions = np.array([
    [-1, 0, 35.0],
    [2,10,35],
    [0,0,35],
])

MinX = [0,float('inf')]
MaxX = [0,float('-inf')]
MinY = [0,float('inf')]
MaxY = [0,float('-inf')]

for id in [0,1,2]:
    x = sample_positions[id][0]
    y = sample_positions[id][1]
    #Determine the Minimum X,Y and Maximum X,Y of the sampling positions
    if (x < MinX[1]):
        MinX = [id, x]
    if (x > MaxX[1]):
        MaxX = [id, x]
    if (y < MinY[1]):
        MinY = [id, y]
    if (y > MaxY[1]):
        MaxY = [id, y]


MinX[1] = calculate_world_coordinates(sample_positions[MinX[0]], camera_offset, 0,0)[0]
MaxX[1] = calculate_world_coordinates(sample_positions[MaxX[0]], camera_offset, IMAGE_SIZE,IMAGE_SIZE)[0]
MinY[1] = calculate_world_coordinates(sample_positions[MinY[0]], camera_offset, 0,0)[1]
MaxY[1] = calculate_world_coordinates(sample_positions[MaxY[0]], camera_offset, IMAGE_SIZE, IMAGE_SIZE)[1]

#Calculate how much the max and min images intersects and determine the needed Array-Width
p1 = MinX[1]
p2 = calculate_world_coordinates(sample_positions[MaxX[0]], camera_offset, 0,0)[0]
p3 = calculate_world_coordinates(sample_positions[MinX[0]], camera_offset, IMAGE_SIZE,IMAGE_SIZE)[0]
t = (p3-p2)/(p3-p1)
arr_width = int(np.ceil(IMAGE_SIZE + IMAGE_SIZE - t * IMAGE_SIZE))

#Calculate how much the max and min images intersects and determine the needed Array-Height
p1 = MinY[1]
p2 = calculate_world_coordinates(sample_positions[MaxY[0]],camera_offset, 0,0)[1]
p3 = calculate_world_coordinates(sample_positions[MinY[0]], camera_offset, IMAGE_SIZE,IMAGE_SIZE)[1]

t = (p3-p2)/(p3-p1)
arr_height = int(np.ceil(IMAGE_SIZE + IMAGE_SIZE - t * IMAGE_SIZE)) 

print(arr_width)
print(arr_height)

(16.0, 16.0)
-17.32076803542495 -14.320768035424951 15.320768035424951
-16.32076803542495 -6.320768035424951 16.32076803542495
560
669


In [2]:
#return percentage of ground visible (depth>34meter)
def check_ground_visibility(depth_image):
    number_visible_ground=0
    for i in depth_image:
        for j in i:
            if j[0]>3400:
                number_visible_ground= number_visible_ground+1
    return number_visible_ground/(512**2)
 
def update_Target_Position():
    if Target_Position[0]<25:
        Target_Position = Target_Position + [0.5,0.5]

In [3]:
def mutate(solution):
    return null

# Waypints in Solution get randomized
def crossover(parent1, parent2):
    return null

Initialization of Drones and Solution Populations

In [4]:
#Spawn Drones and move to initial position

# Create the swarm object by passing the name
# of the world from the .sdf world file.
swarm = Swarm("rhea_swarm")

# Spawn X drones and keep the returning ids as handles
ids = swarm.spawn(NUM_DRONES)

#Create Fake Person Location
person_waypoint =np.array([5,5])

# First waypoints
waypoints = np.empty((0,3),float)
for i in range (NUM_DRONES):
    waypoints= np.append(waypoints,np.array([[math.ceil(i-(NUM_DRONES/2)),0.0,FLIGHT_HEIGHT]]), axis=0)

#create Initial
swarm.waypoints(ids, waypoints)

#Initialize Target Position
Target_Position = [10,10]

Last_Known_Position = Target_Position

#Visibility by Coordiantes File:
grid_visibility = np.full((AREA_SIZE_X,AREA_SIZE_Y), INITIAL_VISIBILITY)

#Create Target Probability Grid
x = np.linspace(-grid_offset, grid_offset, GRID_SIZE)
y = np.linspace(-grid_offset, grid_offset, GRID_SIZE)
x, y = np.meshgrid(x, y)

sigma= 10

prob_density = (1 / (np.sqrt(2 * np.pi)* sigma)) * np.exp(-((x - Last_Known_Position[0])**2 + (y - Last_Known_Position[1])**2) / (2 * sigma**2))


import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3))
plt.contourf(x, y, prob_density, levels=50, cmap='hot')
plt.colorbar(label='Probability Density')
plt.scatter([Last_Known_Position[0]], [Last_Known_Position[1]], color='blue', label='Last Known Position')
plt.scatter([waypoints[:,0]], [waypoints[:,1]], color='green', label='Drones')
plt.show()



NameError: name 'Swarm' is not defined

In [ ]:
#print(swarm.depth_images[1])
print(check_ground_visibility(swarm.depth_images[1]))


In [ ]:
# Problem classes
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.problems.functional import FunctionalProblem
from pymoo.optimize import minimize

# Algorithmss
from pymoode.algorithms import DE, GDE3, NSDE, NSDER
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.nsga2 import NSGA2

# Reference directions and test problems
from pymoo.problems import get_problem
from pymoo.util.ref_dirs import get_reference_directions

# Survival operators
from pymoode.survival import RankAndCrowding, ConstrRankAndCrowding

# Termination
from pymoo.termination.default import DefaultSingleObjectiveTermination, DefaultMultiObjectiveTermination

# Performance
from pymoode.performance import SpacingIndicator


In [ ]:
class MyProblem(Problem):

    def __init__(self, grid_size, n_drones, n_steps, grid_visibility, target_probabilities):
        
        self.grid_size = grid_size
        self.n_drones = n_drones
        n_var=2*n_drones
        #TODO Schrittweite als Var
        self.n_steps = n_steps
        self.grid_visibility = grid_visibility
        self.target_probabilities = target_probabilities

        super().__init__(
            n_var=n_var,  # Variables in the solutions (x,y)
            n_obj=2,      # Number of Objectives we wanna optimize
            n_constr=0,   # No additional constraints, TODO: add no crashing into eachother
            xl=-2.0,      # Lower boundaries of solution variables
            xu=2.0,       # Upper bounds of solution vars
            vtype=int)    

        #x are the waypoints
    def _evaluate(self, x, out, *args, **kwargs):
        
        #reshapes the variables back into a waypoint array
        coordinates = x.reshape(-1, 2)
               
        summed_probability,summed_visibility = 0,0
        
        for waypoint in coordinates:
            x_cord= waypoint[0] + grid_offset
            y_cord= waypoint[1] + grid_offset
            
            summed_probability += prob_density[x_cord][y_cord]
            
            #TODO: nur gemeinsamer Boden
            summed_visibility  += grid_visibility[x_cord][y_cord]
        
        
        out["F"] = np.column_stack([-summed_visibility, -summed_probability])


# Define the problem
problem = MyProblem(GRID_SIZE, NUM_DRONES, NUM_GENERATIONS, grid_visibility, prob_density)

# Define the algorithm
algorithm = DE(
    pop_size=50,
    sampling="real_random",    #Für initiale Population
    variant="DE/rand/1/bin",
    crossover=crossover,
    mutation=mutate,
    #wieviel schritte in zukunft
)





In [ ]:
#termination = get_termination("n_gen", 200)

result = minimize(
    problem,
    algorithm, 
    #termination, 
    seed=1, 
    save_history=True, 
    verbose=True)

#best_solution = result.X[np.argmin(result.F[:, 0] + result.F[:, 1])]

In [ ]:
finished=False
#Drone waypoints got from -25 to 25, Array from 0 to 50

while not finished:
    #wait until all drones recieved
    if swarm.received_frames[ids[-1]]:
        for id in ids:
            print(id)
            drone_waypoint=waypoints[id]
            #rgb_image = swarm.rgb_images[id]
            vis = check_ground_visibility(swarm.depth_images[id])
            
            x_cord=int(drone_waypoint[0]+grid_offset)
            y_cord=int(drone_waypoint[1]+grid_offset)
            
            grid_visibility[x_cord][y_cord]=vis
            #TODO: Check if target was seen, update last known position
            #if TargetSeen:
                #sigma = 1
                #Last_Known_Position = Target_Position

        #Find new best waypoints
        best_solution = result.X
        best_fitness = result.F
        print("Best Solution: {} \n with a Score of {}".format(best_solution,best_fitness))
        
        #Move Drones to best waypoints
        current_waypoints = best_solution
        swarm.waypoints(ids, current_waypoints)
        
        #Update Target Model:
        sigma=sigma+0.5
        prob_density = (1 / (np.sqrt(2 * np.pi)* sigma)) * np.exp(-((x - Last_Known_Position[0])**2 + (y - Last_Known_Position[1])**2) / (2 * sigma**2))
        
        #Move Target-Person
        update_Target_Position()
        print("Moving Target to {}".format(Target_Position))
                       
        
        plt.figure(figsize=(4, 3))
        plt.contourf(x, y, prob_density, levels=50, cmap='hot')
        plt.colorbar(label='Probability Density')
        plt.scatter([Target_Position[0]], [Target_Position[1]], color='blue', label='Target Position')
        plt.scatter([waypoints[:,0]], [waypoints[:,1]], color='green', label='Drones')
        plt.show()
                       
        #if StopCondition:
            #finished=True

In [ ]:
print(grid_visibility)
current_waypoints = waypoints
for d_z in range(0, 30):

    # Set the waypoints for all drones
    current_waypoints = current_waypoints + [[2,0,0],[0,2,0],[0,0,0]]
    
    swarm.waypoints(ids, current_waypoints)
    # Wait until the data has arrived
    time_delta = 0.01           # Delta time per sleep command in seconds
    time_passed = 0.0           # Time counter to keep track of the time in seconds
    timeout = 10.0               # Timeout in case something goes wrong

    timeout_occured = False

    while time_passed < timeout:
        # All frames for a waypoint called send
        # together, so it is enough to check the
        # last one.
        if swarm.received_frames[ids[-1]]:
            print("All frames recieved")
            print (current_waypoints)
            print("depth image for ID={}:".format(1))
            d_i = swarm.depth_images[1]
            print(np.shape(d_i))
            #for i in range(len(d_i)):
            #    print(d_i[i])
            
            for id in ids:
                rgb_image = swarm.rgb_images[id]
                #thermal_image = swarm.thermal_images[id]
                depth_image = swarm.depth_images[id]
                # In this example we only store them in a data directory
                # OpenCV expects BGR but it is RGB, so switch the channels
                #cv2.imwrite(f"../../data/swarm/rgb_image_drone_{id}_dz_{d_z}.png", cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB))    
                #cv2.imwrite(f"../../data/swarm/thermal_image_drone_{id}_dz_{d_z}.png", thermal_image)
                #cv2.imwrite(f"../../data/swarm/depth_image_drone_{id}_dz_{d_z}.png", depth_image)
                drone_waypoint=waypoints[id]
                vis = check_ground_visibility(swarm.depth_images[id])
            
                x_cord=int(drone_waypoint[0]+grid_offset)
                y_cord=int(drone_waypoint[1]+grid_offset)
                print(vis)
                grid_visibility[x_cord][y_cord]=vis
            
            
            plt.figure(figsize=(4, 3))
            plt.contourf(x, y, grid_visibility, levels=50, cmap='hot')
            plt.colorbar(label='Visibility')
            plt.show()
            break
        time.sleep(time_delta)
        time_passed += time_delta
    if time_passed >= timeout:
            timeout_occured = True
            print(f"Timeout reached for waypoints with dz={d_z}")

if timeout_occured:
    raise TimeoutError("Timeout occured while waiting for waypoint")
    
print("Finished")